In [3]:
from xgboost import XGBRegressor

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score


In [20]:
df = pd.read_excel("../customer.xlsx", sheet_name="train")

In [21]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()
df['zone_encode'] = label_encoder.fit_transform(df['Zone'])

In [99]:
X = df[
    [
        "All Customers",
        "New Customer",
        #"Old Customer",
        "Customer Lost",
        #"Growth",
        #"Growth (Market)",
        #"zone_encode"
    ]
]
y = df[["Sales Value"]]

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((78, 3), (34, 3))

In [101]:
regressor = XGBRegressor()

In [102]:
#fit the model
#Training data is in X_train and the corresponding price value is in the Y_train
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [103]:
sales_data_prediction = regressor.predict(X_train)

In [104]:
# In order to check the performance of the model we find the R squared Value
r2_sales = r2_score(y_train, sales_data_prediction)
print('R Squared value = ', r2_sales)

R Squared value =  0.90555027999109


In [105]:
data_prediction = regressor.predict(X_test)
r2_sales = r2_score(y_test, data_prediction)
print('R Squared value = ', r2_sales)

R Squared value =  0.7849327120641519


In [106]:
df2 = pd.read_excel("../customer.xlsx", sheet_name="predict")

In [107]:
zones_to_delete = ["Africa", "South/Latin America", "North America"]
# If we sold only visited zones
df3 = df2[~df2["Zone"].isin(zones_to_delete)]

In [108]:
df3["zone_encode"] = label_encoder.fit_transform(df3["Zone"])
df2["zone_encode"] = label_encoder.fit_transform(df2["Zone"])

C:\Users\suwatsara_d\AppData\Local\Temp\ipykernel_4616\2141329738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["zone_encode"] = label_encoder.fit_transform(df3["Zone"])


In [109]:
X1 = df3[
    [
        "All Customers",
        "New Customer",
        #"Old Customer",
        "Customer Lost",
        #"Growth",
        #"Growth (Market)",
        #"zone_encode"
    ]
]

# If we sold only visited zones and new zones
X2 = df2[
    [
        "All Customers",
        "New Customer",
        #"Old Customer",
        "Customer Lost",
        #"Growth",
        #"Growth (Market)",
        #"zone_encode"
    ]
]

In [110]:
y_pred = regressor.predict(X1)
df3["y_pred"] = y_pred
predict_1 = df3.groupby(["Year", "Zone"])["y_pred"].sum()
# Convert the grouped object to a DataFrame
predict_1 = predict_1.reset_index()
predict_1.groupby(["Year"])["y_pred"].sum()

C:\Users\suwatsara_d\AppData\Local\Temp\ipykernel_4616\2818164596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["y_pred"] = y_pred


Year
2024    6.088500e+09
2025    8.265601e+09
2026    9.407166e+09
2027    7.990962e+09
2028    1.073589e+10
2029    9.496656e+09
2030    1.098470e+10
Name: y_pred, dtype: float32

In [111]:
y_pred = regressor.predict(X2)
df2["y_pred"] = y_pred
predict_1 = df2.groupby(["Year", "Zone"])["y_pred"].sum()
# Convert the grouped object to a DataFrame
predict_1 = predict_1.reset_index()
predict_1.groupby(["Year"])["y_pred"].sum()

Year
2024    6.102031e+09
2025    9.156029e+09
2026    9.917862e+09
2027    8.127063e+09
2028    1.332061e+10
2029    1.261572e+10
2030    1.401677e+10
Name: y_pred, dtype: float32